# Train–Test-Split

Ziel: Wir erstellen einen zeitbasierten Train–Test-Split, um die Wahrscheinlichkeit eines F1-Einstiegs basierend auf Performance bis Saison t zu modellieren.

Prinzip:
- Training nutzt frühere Jahre
- Test nutzt spätere Jahre
- Keine zufällige Durchmischung über Jahre


In [1]:
#Imports und Pfad robust setzen
from pathlib import Path
import pandas as pd

def find_project_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / "data").exists() and (p / "src").exists():
            return p
    return start

PROJECT_ROOT = find_project_root(Path.cwd())
DATA_PATH = PROJECT_ROOT / "data/model_input/f2_f3_features_with_f1_label.csv"

print("Project root:", PROJECT_ROOT)
print("Data path:", DATA_PATH)
print("Exists:", DATA_PATH.exists())


Project root: /Users/sheyla/Desktop/rookie_invest_ML
Data path: /Users/sheyla/Desktop/rookie_invest_ML/data/model_input/f2_f3_features_with_f1_label.csv
Exists: True


In [2]:
#Daten laden und Basis prüfen
df = pd.read_csv(DATA_PATH)

print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
print(df["series"].value_counts() if "series" in df.columns else "No series column")
print(df["f1_entry"].value_counts())
df.head(5)


Shape: (488, 28)
Columns: ['series', 'year', 'driver_name', 'driver_code', 'team_name', 'n_races', 'total_points', 'avg_points', 'avg_finish', 'best_finish', 'worst_finish', 'wins', 'win_rate', 'podiums', 'podium_rate', 'points_finishes', 'points_rate', 'top10_finishes', 'top10_rate', 'total_laps', 'avg_kph', 'finish_std', 'points_std', 'dnf_count', 'dnf_rate', 'avg_best_lap_s', 'first_f1_year', 'f1_entry']
series
F2    244
F3    244
Name: count, dtype: int64
f1_entry
False    456
True      32
Name: count, dtype: int64


,series,year,driver_name,driver_code,team_name,n_races,total_points,avg_points,avg_finish,best_finish,...,top10_rate,total_laps,avg_kph,finish_std,points_std,dnf_count,dnf_rate,avg_best_lap_s,first_f1_year,f1_entry
0,F2,2017,A Albon,ALB,ART Grand Prix,10,67.0,6.70,7.700000,2,...,0.900000,245.0,177.766000,4.498148,6.429965,0.0,0.000000,97.620800,2005.0,False
1,F2,2017,R Binder,BIN,Rapax,1,0.0,0.00,17.000000,17,...,0.000000,28.0,169.580000,NaN,NaN,0.0,0.000000,89.032000,NaN,False
2,F2,2017,R Boschung,BOS,Campos Racing,10,2.0,0.20,16.200000,9,...,0.100000,195.0,174.463500,3.457681,0.632456,5.0,0.500000,98.379100,NaN,False
3,F2,2017,S Canamasas,CAN,Trident,7,14.0,2.00,12.428571,4,...,0.285714,170.0,172.147143,5.223573,4.472136,1.0,0.142857,96.403857,NaN,False
4,F2,2017,J Cecotto,CEC,Rapax,4,21.0,5.25,9.000000,2,...,0.750000,100.0,166.890500,5.354126,8.539126,0.0,0.000000,100.324250,NaN,False


## Zeitachse prüfen

Wir prüfen die Verteilung über die Jahre und wie viele positive Fälle pro Jahr vorhanden sind.
Das ist entscheidend, damit der Test-Split nicht zufällig nur 0 oder 1 positive Beispiele enthält.


In [3]:
#Jahresverteilung und Positives pro Jahr
print("Year range:", df["year"].min(), "-", df["year"].max())

year_counts = df["year"].value_counts().sort_index()
year_pos = df.groupby("year")["f1_entry"].sum().sort_index()

summary = pd.DataFrame({
    "n_rows": year_counts,
    "n_positive": year_pos,
})
summary["positive_share"] = summary["n_positive"] / summary["n_rows"]

summary


Year range: 2017 - 2025


,n_rows,n_positive,positive_share
year,,,
2017,28,6,0.214286
2018,24,5,0.208333
2019,60,3,0.050000
2020,59,6,0.101695
2021,63,5,0.079365
2022,67,4,0.059701
2023,59,3,0.050847
2024,64,0,0.000000
2025,64,0,0.000000


## Wahl des Cutoff-Jahres

Wir wählen das Jahr 2021 als Cutoff.

Begründung:
- Die Testjahre 2022 und 2023 enthalten mehrere positive Fälle
- Die Testperiode ist zeitlich neuer als das Training
- Jahre ohne beobachtbare F1-Einstiege (2024–2025) werden ausgeschlossen, weil Fahrer aus diesen Saisons noch nicht ausreichend Zeit hatten, um in die Formel 1 aufzusteigen, wodurch das Zielmerkmal systematisch verzerrt wäre.


In [4]:
CUTOFF = 2021

train_df = df[df["year"] <= CUTOFF].copy()
test_df = df[(df["year"] > CUTOFF) & (df["year"] <= 2023)].copy()

print("Train:", train_df.shape, "Positives:", int(train_df["f1_entry"].sum()))
print("Test :", test_df.shape,  "Positives:", int(test_df["f1_entry"].sum()))

print("Train years:", train_df["year"].min(), "-", train_df["year"].max())
print("Test years :", test_df["year"].min(), "-", test_df["year"].max())


Train: (234, 28) Positives: 25
Test : (126, 28) Positives: 7
Train years: 2017 - 2021
Test years : 2022 - 2023


## Speichern der finalen Splits

Um die Reproduzierbarkeit sicherzustellen, speichern wir die finalen Trainings- und Testdaten separat ab.
Diese Dateien werden im weiteren Projektverlauf direkt für das Modelltraining verwendet.


In [5]:
#Split speichern
OUT_DIR = PROJECT_ROOT / "data/model_input/splits"
OUT_DIR.mkdir(parents=True, exist_ok=True)

train_path = OUT_DIR / f"train_upto_{CUTOFF}.csv"
test_path = OUT_DIR / f"test_after_{CUTOFF}.csv"

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

print("Saved train:", train_path)
print("Saved test :", test_path)


Saved train: /Users/sheyla/Desktop/rookie_invest_ML/data/model_input/splits/train_upto_2021.csv
Saved test : /Users/sheyla/Desktop/rookie_invest_ML/data/model_input/splits/test_after_2021.csv
